In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

TODO
Build an image classifier with resnet18 or any other arch to classify between real dog and cartoon dog
Steps:
1. Setup data - download the data, preview data files
2. Define dataloader
3. Define learner model
4. Train the model (simple)
5. Clean the data
6. Train the model full
7. Test the model
8. Save the model

In [ ]:
# install
!pip install -U duckduckgo_search

In [ ]:
# imports
from fastcore.all import *
from fastai.vision.all import *
from duckduckgo_search import DDGS

import time

In [ ]:
# search images
def search_images(topic, mx=100):
    urls = L(DDGS().images(topic, max_results=mx)).itemgot('image')
    return urls

In [ ]:
# checking out data
realimage = search_images('real dog', 1)
download_images("", urls=realimage)
cartoonimage = search_images('cartoon dog', 1)
download_images("", urls=cartoonimage)

In [ ]:
Path().ls()

In [ ]:
realimage

In [ ]:
# checking out data
rimg = 'de918370-d2ce-479c-8d07-489a80fcdb73.jpg'
im = Image.open(rimg)
im

In [ ]:
# checking out data
rimg = 'c28def76-4125-4086-b0ab-ea2d5218903d.jpg'
im = Image.open(rimg)
im

In [ ]:
topics = "real dog", "cartoon dog"
path = Path('dog_real_cartoon')

for topic in topics:
    dest = path/topic
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(topic))
    time.sleep(5)
    resize_images(dest, max_size=400, dest=dest)


In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(blocks=(ImageBlock, CategoryBlock),
                get_items=get_image_files,
                get_y=parent_label,
                splitter=RandomSplitter(seed=32),
                item_tfms=[Resize(192, method='squish')]
               ).dataloaders(path, bs=32)


In [ ]:
dls.show_batch(max_n=6, nrows=1)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)

In [ ]:
learn.fine_tune(3)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)


In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5)